<a href="https://colab.research.google.com/github/santiagoruizs/IML_Proyecto_Final/blob/master/colabs/housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O housing_train.csv 'https://raw.githubusercontent.com/sergiakalorth/MachineLearning-CIM-UPC/master/Final%20Project/datasets/Ames%20Housing%20Price/train.csv'
!wget -O housing_test.csv 'https://raw.githubusercontent.com/sergiakalorth/MachineLearning-CIM-UPC/master/Final%20Project/datasets/Ames%20Housing%20Price/test.csv'

--2020-07-23 11:29:32--  https://raw.githubusercontent.com/sergiakalorth/MachineLearning-CIM-UPC/master/Final%20Project/datasets/Ames%20Housing%20Price/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460676 (450K) [text/plain]
Saving to: ‘housing_train.csv’

housing_train.csv   100%[===================>] 449.88K  --.-KB/s    in 0.06s   

2020-07-23 11:29:32 (6.91 MB/s) - ‘housing_train.csv’ saved [460676/460676]

--2020-07-23 11:29:35--  https://raw.githubusercontent.com/sergiakalorth/MachineLearning-CIM-UPC/master/Final%20Project/datasets/Ames%20Housing%20Price/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com

In [124]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/housing_train.csv')
train.head ()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test= pd.read_csv('/content/housing_test.csv')
test.head ()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


Para hacer el merge de ambos datasets, incluimos la columna de SalePrice en el dataset de test, al final de la limpieza, excluiremos SalePrice de la tabla concatenada.

In [4]:
test['SalePrice']=train['SalePrice']
test.head ()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,208500
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,181500
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,223500
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,...,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,140000
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,...,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,250000


In [ ]:
train.shape

(1460, 81)

In [ ]:
test.shape

(1459, 81)

In [5]:
df = train.append(test, sort = False,ignore_index=True)

print(df.shape)
df.head ()

(2919, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,...,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,...,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,...,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,...,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,...,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Procedemos a limpiar los Nans.

In [6]:
totalt = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(34)



,Total,Percent
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
FireplaceQu,1420,0.486468
LotFrontage,486,0.166495
GarageCond,159,0.054471
GarageYrBlt,159,0.054471
GarageFinish,159,0.054471
GarageQual,159,0.054471


#Para rellenar estos Nans que no tienen calidad de piscina asignada nos basamos en las medidas de poolArea, las que tienen PoolArea 0 las dejamos en NA porque no hay piscina y las que sí tienen area las comparamos con las medidas promedio.

In [7]:
df.loc[df.PoolArea != 0].PoolQC.describe()

count     10
unique     3
top       Gd
freq       4
Name: PoolQC, dtype: object

In [8]:
df.loc[(df.PoolArea != 0)&(df.PoolQC.isnull()), 'PoolQC'] = 'Ex'

In [9]:
df.PoolQC = df.PoolQC.fillna('NA')

#Para rellenar los features adicionales en las propiedades, nos basamos en el valor monetario de éstas. Si no tenian valor, pues no existe tal atributo extra.

In [10]:
df.loc[(df.MiscVal == 0)&(~df.MiscFeature.isnull())]


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
873,874,40,RL,60.0,12144,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1949,1950,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,CBlock,TA,TA,No,Rec,375.0,Unf,0.0,457.0,832.0,GasA,...,Y,SBrkr,1036,0,232,1268,0.0,0.0,1,0,3,1,TA,6,Typ,1,Gd,Attchd,1949.0,Unf,1.0,288.0,TA,TA,Y,0,28,0,0,0,0,NA,NaN,Othr,0,9,2009,WD,Normal,133000
1200,1201,20,RL,71.0,9353,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,4,5,1970,1970,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,864.0,864.0,GasA,...,Y,SBrkr,864,0,0,864,0.0,0.0,1,0,3,1,TA,5,Typ,0,NaN,Attchd,1972.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NA,NaN,Shed,0,7,2006,Oth,Abnorml,116050
2431,2432,20,RL,NaN,7500,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1959,2003,Hip,CompShg,BrkFace,BrkFace,None,0.0,TA,TA,CBlock,TA,TA,No,GLQ,574.0,Unf,0.0,466.0,1040.0,GasA,...,Y,SBrkr,1040,0,0,1040,1.0,0.0,1,0,3,1,Gd,6,Typ,0,NaN,Attchd,1959.0,RFn,1.0,286.0,TA,TA,Y,0,0,0,0,0,0,NA,NaN,Shed,0,7,2007,WD,Normal,173000


In [11]:
df.MiscFeature = df.MiscFeature.fillna('NA')

#La misma lógica de las piscinas, las repetimos con las chimeneas. Si no hay chimenea, NA

In [12]:
df.loc[(df.Fireplaces == 0)&(df.FireplaceQu.isnull())]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,...,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NA,NaN,NA,0,2,2008,WD,Normal,208500
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732.0,Unf,0.0,64.0,796.0,GasA,...,Y,SBrkr,796,566,0,1362,1.0,0.0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2.0,480.0,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000
10,11,20,RL,70.0,11200,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1965,1965,Hip,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,906.0,Unf,0.0,134.0,1040.0,GasA,...,Y,SBrkr,1040,0,0,1040,1.0,0.0,1,0,3,1,TA,5,Typ,0,NaN,Detchd,1965.0,Unf,1.0,384.0,TA,TA,Y,0,0,0,0,0,0,NA,NaN,NA,0,2,2008,WD,Normal,129500
12,13,20,RL,NaN,12968,Pave,NaN,IR2,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1962,1962,Hip,CompShg,HdBoard,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,737.0,Unf,0.0,175.0,912.0,GasA,...,Y,SBrkr,912,0,0,912,1.0,0.0,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1962.0,Unf,1.0,352.0,TA,TA,Y,140,0,0,0,176,0,NA,NaN,NA,0,9,2008,WD,Normal,144000
15,16,45,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,7,8,1929,2001,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,832.0,832.0,GasA,...,Y,FuseA,854,0,0,854,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1991.0,Unf,2.0,576.0,TA,TA,Y,48,112,0,0,0,0,NA,GdPrv,NA,0,7,2007,WD,Normal,132000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,2913,160,RM,21.0,1533,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,408.0,Unf,0.0,138.0,546.0,GasA,...,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,0,0,0,0,0,NA,NaN,NA,0,12,2006,WD,Abnorml,145000
2913,2914,160,RM,21.0,1526,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,...,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,34,0,0,0,0,NA,GdPrv,NA,0,6,2006,WD,Normal,84500
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,...,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NA,NaN,NA,0,6,2006,WD,Normal,185000
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5

In [13]:
df.FireplaceQu = df.FireplaceQu.fillna('NA')

#Con lo siguiente podemos reemplazar los nans de estos valores clasificándolos con la información de la venta y obtenemos que el valor más repetido con los valores de MSSubClass es RL.

In [14]:
df.loc[(df.MSSubClass==30)|(df.MSSubClass==20)|(df.MSSubClass==70)].MSZoning.describe()

count     1342
unique       5
top         RL
freq      1134
Name: MSZoning, dtype: object

In [15]:
df.MSZoning = df.MSZoning.fillna('RL')

#Con esto vemos que solo hay 2 NaN y que el resto de valores son todos AllPub a excepción de uno que es NoSeWa, por esto se reemplazan los valores con AllPub.

In [16]:
df.Utilities.describe()

count       2917
unique         2
top       AllPub
freq        2916
Name: Utilities, dtype: object

In [17]:
df.Utilities = df.Utilities.fillna('AllPub')

#y así sucesivamente, rellenamos los nans de las demás features.

In [18]:
df.Fence = df.Fence.fillna('NA')
df.Alley = df.Alley.fillna('NA')

In [19]:
df.loc[(df.MasVnrArea==198)].MasVnrType.describe()

count         5
unique        2
top       Stone
freq          3
Name: MasVnrType, dtype: object

In [20]:
df.loc[(df.MasVnrArea==198)&(df.MasVnrType.isnull()),'MasVnrType'] = 'Stone'

#Como se ve en las siguientes búsquedas, no existen atributos NA para estas columnas, y dado que son elementos que no están presentes en la mayoría de las casas y viendo la cantidad de Nans en sus variables podríamos asumir que estas pueden ser NA.

In [21]:
df.loc[df.GarageArea == 0, 'GarageCond'] = 'NA'
df.loc[df.GarageArea == 0, 'GarageQual'] = 'NA'
df.loc[df.GarageArea == 0, 'GarageFinish'] = 'NA'
df.loc[df.GarageArea == 0, 'GarageYrBlt'] = 'NA'
df.loc[df.GarageArea == 0, 'GarageType'] = 'NA'
df.loc[df.TotalBsmtSF == 0, 'BsmtQual'] = 'NA'
df.loc[df.TotalBsmtSF == 0, 'BsmtCond'] = 'NA'
df.loc[df.TotalBsmtSF == 0, 'BsmtExposure'] = 'NA'
df.loc[df.TotalBsmtSF == 0, 'BsmtFinType1'] = 'NA'
df.loc[df.TotalBsmtSF == 0, 'BsmtFinType2'] = 'NA'

#Las features que quedan con Nans, usaremos el encoder para su imputación. El resto (númericas) usaremos el KNN.

In [22]:
totalt = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(15)


,Total,Percent
LotFrontage,486,0.166495
MasVnrArea,23,0.007879
MasVnrType,23,0.007879
BsmtCond,4,0.001370
BsmtExposure,4,0.001370
BsmtQual,3,0.001028
GarageCond,2,0.000685
BsmtFinType2,2,0.000685
BsmtFullBath,2,0.000685
BsmtHalfBath,2,0.000685


In [23]:
col = df.select_dtypes(include = 'object').columns

In [24]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in col:
  le.fit(df[i].astype(str))  
  df[i] = le.transform(df[i].astype(str))

In [25]:
totalt = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
LotFrontage,486,0.166495
MasVnrArea,23,0.007879
BsmtFullBath,2,0.000685
BsmtHalfBath,2,0.000685
BsmtUnfSF,1,0.000343
GarageCars,1,0.000343
GarageArea,1,0.000343
TotalBsmtSF,1,0.000343
BsmtFinSF1,1,0.000343
BsmtFinSF2,1,0.000343


In [26]:
from sklearn.impute import KNNImputer
import io
imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputed_data = imputer.fit_transform(df)

df_imputed = pd.DataFrame(imputed_data,columns=df.columns)
df_imputed.loc[df['LotFrontage'].isna()==True]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
7,8.0,60.0,3.0,66.5,10382.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,14.0,4.0,2.0,0.0,5.0,7.0,6.0,1973.0,1973.0,1.0,1.0,6.0,6.0,3.0,240.0,3.0,4.0,1.0,2.0,4.0,2.0,0.0,859.0,1.0,32.0,216.0,1107.0,1.0,...,1.0,4.0,1107.0,983.0,0.0,2090.0,1.0,0.0,2.0,1.0,3.0,1.0,3.0,7.0,6.0,2.0,5.0,1.0,64.0,2.0,2.0,484.0,5.0,5.0,2.0,235.0,204.0,228.0,0.0,0.0,0.0,3.0,4.0,3.0,350.0,11.0,2009.0,8.0,4.0,200000.0
12,13.0,20.0,3.0,70.0,12968.0,1.0,1.0,1.0,3.0,0.0,4.0,0.0,19.0,2.0,2.0,0.0,2.0,5.0,6.0,1962.0,1962.0,3.0,1.0,6.0,10.0,2.0,0.0,3.0,4.0,1.0,4.0,4.0,4.0,0.0,737.0,6.0,0.0,175.0,912.0,1.0,...,1.0,4.0,912.0,0.0,0.0,912.0,1.0,0.0,1.0,0.0,2.0,1.0,3.0,4.0,6.0,0.0,3.0,5.0,53.0,3.0,1.0,352.0,5.0,5.0,2.0,140.0,0.0,0.0,0.0,176.0,0.0,3.0,4.0,1.0,0.0,9.0,2008.0,8.0,4.0,144000.0
14,15.0,20.0,3.0,79.5,10920.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,12.0,2.0,2.0,0.0,2.0,6.0,5.0,1960.0,1960.0,3.0,1.0,8.0,8.0,1.0,212.0,3.0,4.0,1.0,4.0,4.0,4.0,1.0,733.0,6.0,0.0,520.0,1253.0,1.0,...,1.0,4.0,1253.0,0.0,0.0,1253.0,1.0,0.0,1.0,1.0,2.0,1.0,3.0,5.0,6.0,1.0,1.0,1.0,51.0,2.0,1.0,352.0,5.0,5.0,2.0,0.0,213.0,176.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,5.0,2008.0,8.0,4.0,157000.0
16,17.0,20.0,3.0,89.0,11241.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,12.0,2.0,2.0,0.0,2.0,6.0,7.0,1970.0,1970.0,1.0,1.0,13.0,14.0,1.0,180.0,3.0,4.0,1.0,4.0,4.0,4.0,0.0,578.0,6.0,0.0,426.0,1004.0,1.0,...,1.0,4.0,1004.0,0.0,0.0,1004.0,1.0,0.0,1.0,0.0,2.0,1.0,3.0,5.0,6.0,1.0,5.0,1.0,61.0,0.0,2.0,480.0,5.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,3.0,700.0,3.0,2010.0,8.0,4.0,149000.0
24,25.0,20.0,3.0,62.5,8246.0,1.0,1.0,0.0,3.0,0.0,4.0,0.0,19.0,2.0,2.0,0.0,2.0,5.0,8.0,1968.0,2001.0,1.0,1.0,9.0,10.0,2.0,0.0,3.0,2.0,1.0,4.0,4.0,2.0,5.0,188.0,0.0,668.0,204.0,1060.0,1.0,...,1.0,4.0,1060.0,0.0,0.0,1060.0,1.0,0.0,1.0,0.0,3.0,1.0,2.0,6.0,6.0,1.0,5.0,1.0,59.0,3.0,1.0,270.0,5.0,5.0,2.0,406.0,90.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.0,5.0,2010.0,8.0,4.0,154000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,2848.0,20.0,3.0,79.0,11088.0,1.0,1.0,3.0,3.0,0.0,0.0,0.0,5.0,2.0,2.0,0.0,2.0,8.0,5.0,2002.0,2002.0,3.0,1.0,11.0,12.0,2.0,0.0,2.0,4.0,2.0,2.0,4.0,4.0,2.0,872.0,6.0,0.0,476.0,1348.0,1.0,...,1.0,4.0,1358.0,0.0,0.0,1358.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,5.0,6.0,1.0,5.0,1.0,93.0,3.0,2.0,418.0,5.0,5.0,2.0,68.0,166.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.0,6.0,2006.0,8.0,4.0,136000.0
2850,2851.0,60.0,3.0,73.0,21533.0,1.0,1.0,1.0,3.0,0.0,2.0,0.0,5.0,1.0,2.0,0.0,5.0,7.0,5.0,1996.0,1997.0,1.0,1.0,12.0,13.0,2.0,0.0,2.0,4.0,2.0,2.0,4.0,4.0,6.0,0.0,6.0,0.0,1065.0,1065.0,1.0,...,1.0,4.0,1065.0,984.0,0.0,2049.0,0.0,0.0,2.0,1.0,4.0,1.0,2.0,9.0,6.0,1.0,5.0,1.0,88.0,3.0,2.0,467.0,5.0,5.0,2.0,120.0,48.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.0,8.0,2006.0,8.0,4.0,235000.0
2900,2901.0,20.0,3.0,90.0,50102.0,1.0,1.0,0.0,2.0,0.0,4.0,1.0,23.0,2.0,2.0,0.0,2.0,6.0,5.0,1958.0,1958.0,1.0,5.0,9.0,10.0,2.0,0.0,3.0,4.0,1.0,2.0,4.0,1.0,1.0,909.0,6.0,0.0,723.0,1632.0,1.0,...,1.0,4.0,1650.0,0.0,0.0,165

In [27]:
df_imputed.isnull().sum ()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [ ]:
df_imputed.shape

(2919, 81)

#Normalización y Split Data

In [ ]:
from sklearn.preprocessing import StandardScaler
target = train['SalePrice'].values

# select_features = [''MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street']
select_features = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition']


scaler = StandardScaler()
dfScaled = scaler.fit_transform(df_imputed[select_features])

train = dfScaled[0:1460].copy()
test = dfScaled[1459:].copy()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

#Modeling

#Regresión Linear

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

clf = LinearRegression(normalize=True)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

print("El R2 es: {}".format(r2_score(y_test,y_pred)))
print("El pendiente es: {}".format(clf.coef_))
print("El intercept es: {}".format(clf.intercept_))

El R2 es: 0.8443824993533948
El pendiente es: [-5.20705878e+03 -1.16945188e+03 -4.83347088e+03  2.84291615e+03
  1.40689293e+03  8.78318007e+02 -1.80356066e+03  2.17226587e+03
 -9.48225539e+02  1.49503236e+02  2.97847515e+03  2.64451665e+03
 -4.17808667e+02 -2.16373738e+03 -3.56675365e+03 -2.19357573e+03
  1.51644895e+04  5.91710133e+03  6.22010527e+03  2.12754401e+02
  1.99296640e+03  2.83146185e+03 -4.24589659e+03  2.47259312e+03
  2.13674857e+03  4.87373228e+03 -6.06894032e+03  1.56116244e+01
  4.48008412e+02 -6.47980671e+03  2.40979991e+03 -4.23213336e+03
 -1.13054240e+03  5.01668174e+16  1.85464412e+03  1.86260594e+16
  4.83938858e+16 -4.85187230e+16 -3.38999482e+02 -9.42521808e+01
  4.74861322e+02 -3.90657973e+02  3.59275474e+16  3.92550471e+16
  4.24843328e+15 -4.63377424e+16  4.13482938e+03 -6.34292559e+02
  1.13371085e+03 -2.78457575e+02 -1.78612420e+03 -2.78501983e+03
 -7.78522376e+03  6.56851691e+03  3.96390579e+03  3.97981561e+03
 -1.54497397e+03  2.96630018e+01  1.38022353

#RandomForest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


regressor = RandomForestRegressor(n_estimators=400, random_state=20)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

print("El R2 es: {}".format(r2_score(y_test,y_pred)))




El R2 es: 0.8934875624644986


#Simplicando las features.




#Eliminación Recursiva de Features

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE

y= df_imputed ['SalePrice']


svc = SVC(kernel="linear")
rfe = RFE(estimator=svc, n_features_to_select=10, step=1)
rfe.fit(dfScaled, y)
print('Kept features: {}'.format(rfe.support_))
print('Ranking of the features: {}'.format(rfe.ranking_))

Kept features: [False False False False False False  True False False False False  True
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
  True False False False False False False False False False  True False
 False  True False False False False False  True False False False False
 False  True False False False False False False False False False False
 False False False  True  True False False]
Ranking of the features: [41 18 27 60 68 49  1 37 70  6 51  1 32 64 24 16 43 19 55 23 11 61  4 31
  3 25 33 26  1 28 29  7  9 20 42 53  1 39 65  8 57 36 48 22 63 50  1 40
  5  1  2 58 17 30 56  1 14 15 45 10 38  1 47 59 54 12 13 46 62 35 69 67
 21 52 66  1  1 44 34]


In [28]:
from sklearn.preprocessing import StandardScaler #Las features están rankiadas por orden según con el RFE utilizando las 50 más importantes
target = train['SalePrice'].values

# select_features = [''MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street']

select_features = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street','Alley', 'LotShape', 'LandContour', 'Utilities','LandSlope', 'Neighborhood', 'BldgType', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
'MasVnrArea', 'ExterQual',  'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating','HeatingQC', 'CentralAir', 'Electrical',  '2ndFlrSF','GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath','HalfBath', 'BedroomAbvGr', 'KitchenQual','TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 
'GarageCars', 'GarageArea', 'GarageQual','GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF','EnclosedPorch','PoolArea', 'PoolQC','Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'SaleType']



scaler = StandardScaler()
dfScaled = scaler.fit_transform(df_imputed[select_features])

train = dfScaled[0:1460].copy()
test = dfScaled[1459:].copy()

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

#Regresión Linear

In [30]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

clf = LinearRegression(normalize=True)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

print("El R2 es: {}".format(r2_score(y_test,y_pred)))
print("El pendiente es: {}".format(clf.coef_))
print("El intercept es: {}".format(clf.intercept_))

El R2 es: 0.8394000500609317
El pendiente es: [-7.56124718e+03 -9.89191203e+02 -5.06320046e+03  2.83278857e+03
  1.26986062e+03  2.53347995e+02 -1.91602155e+03  2.19217107e+03
 -9.59704033e+02  3.19163770e+03  2.40061049e+03 -2.94802849e+03
  1.74172171e+04  5.45927416e+03  8.56508287e+03  6.92344681e+02
  2.15322901e+03  2.85425897e+03 -3.39616470e+03  2.07416329e+03
  2.65516937e+03  4.73512385e+03 -6.59912036e+03  6.51367188e+02
  2.30701705e+03 -4.78514787e+03 -7.79749452e+02 -1.10607825e+17
 -4.10667454e+16 -1.06698865e+17  1.06974106e+17 -2.15058523e+02
 -5.43676984e+02  6.12921025e+02 -4.36511307e+02 -6.65876558e+02
  2.23658201e+04  4.06738657e+03 -7.63228537e+02  2.14766440e+03
 -2.41161515e+02 -3.12471600e+03 -8.52651662e+03  5.09634765e+03
  3.45627952e+03  4.81768229e+03 -1.75430191e+03  1.00424152e+04
 -1.04722572e+03 -2.16783534e+03  6.34131016e+01  5.07056851e+02
  2.47947014e+03  3.28142372e+01 -9.96307065e+02 -1.17796613e+04
 -1.55528998e+04  1.01056593e+03 -7.75759208

#RandomForest Regressor

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


regressor = RandomForestRegressor(n_estimators=500, random_state=20)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

print("El R2 es: {}".format(r2_score(y_test,y_pred)))


El R2 es: 0.8971798694757777


#Consumo Energía

In [132]:
import pandas as pd
dfe = pd.read_excel('/content/ENB2012_data.xlsx',header=0)
dfe.head ()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [133]:
dfe.isnull().sum()

X1    0
X2    0
X3    0
X4    0
X5    0
X6    0
X7    0
X8    0
Y1    0
Y2    0
dtype: int64

In [134]:
X = dfe.drop(columns =['Y1', 'Y2'])
Y1 = dfe['Y1']
Y2 = dfe['Y2']

In [136]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)

In [137]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(X_sc, Y1, test_size=0.4, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(X_sc, Y2, test_size=0.4, random_state=42)

In [138]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

clf1 = LinearRegression(normalize=True)
clf2 = LinearRegression(normalize=True)
clf1.fit(x_train1,y_train1)
clf2.fit(x_train2,y_train2)
y_pred1 = clf1.predict(x_test1)
y_pred2 = clf2.predict(x_test2)
print("El R2 de 1 es: {}".format(r2_score(y_test1,y_pred1)))
print("El pendiente 1 es: {}".format(clf1.coef_))
print("El intercept 1 es: {}".format(clf1.intercept_))
print("El R2 de 2 es: {}".format(r2_score(y_test2,y_pred2)))
print("El pendiente 2 es: {}".format(clf2.coef_))
print("El intercept 2 es: {}".format(clf2.intercept_))

El R2 de 1 es: 0.9127543795250956
El pendiente 1 es: [-6.33875164e+00 -5.63080313e+13  2.78877239e+13  5.77436230e+13
  7.23918441e+00  1.49578122e-03  2.59580830e+00  2.05825495e-01]
El intercept 1 es: 22.135879298308435
El R2 de 2 es: 0.8929068216824306
El pendiente 2 es: [-7.41741250e+00 -1.17723078e+14  5.83048035e+13  1.20724467e+14
  7.33827147e+00  1.77238569e-01  1.72079742e+00 -2.22531367e-01]
El intercept 2 es: 24.39366171145923


#Comparé las zonas por clase social en el site de Sunroof ya que te muestra el ingreso familiar por área, y concluí que el consumo kW de paneles solares va por square feets del techo de la propiedad independientemente de la zona en la que esté y la zona en la que le de el sol la mayor parte del día.

#Incluí la columna con el estimado de consumo en los datasets de entreno y la concatenada después de limpiar nans.

In [125]:
conditionlist = [
  (train ['2ndFlrSF'] != 0) &   (train ['1stFlrSF'] < 560) &  (train ['2ndFlrSF']  < 560),
  (train['2ndFlrSF'] != 0) &   (train ['1stFlrSF'] > 560) & (train ['1stFlrSF'] <= 600) &  (train ['2ndFlrSF'] > 560) & (train ['2ndFlrSF'] <= 600),
  (train['2ndFlrSF'] != 0) &   (train ['1stFlrSF'] > 600) & (train ['1stFlrSF'] <= 650) &  (train['2ndFlrSF'] > 600) & (train ['2ndFlrSF'] <= 650),
  (train['2ndFlrSF'] != 0) &   (train ['1stFlrSF'] > 650) & (train ['1stFlrSF'] <= 750) &  (train ['2ndFlrSF'] > 650) & (train ['2ndFlrSF'] <= 750),
   (train['2ndFlrSF']!= 0) &  (train ['1stFlrSF'] >= 750) & (train ['2ndFlrSF'] >= 750)
     ]
choicelist = [7, 8, 9, 10, 11]
train['Consumo kW'] = np.select(conditionlist, choicelist, default= '0')

In [126]:
conditionlist2 = [
  (df_imputed['2ndFlrSF'] != 0) &   (df_imputed ['1stFlrSF'] <= 560.0) &  (df_imputed ['2ndFlrSF']  <= 560.0),
  (df_imputed['2ndFlrSF'] != 0) &   (df_imputed ['1stFlrSF'] > 560.0) & (df_imputed ['1stFlrSF'] <= 600.0) &  (df_imputed ['2ndFlrSF'] > 560.0) &(df_imputed ['2ndFlrSF'] <= 600.0),
  (df_imputed['2ndFlrSF'] != 0) &   (df_imputed ['1stFlrSF'] > 600.0) & (df_imputed ['1stFlrSF'] <= 650.0) &  (df_imputed ['2ndFlrSF'] > 600.0) &(df_imputed ['2ndFlrSF'] <= 650.0),
  (df_imputed['2ndFlrSF'] != 0) &   (df_imputed ['1stFlrSF'] > 650.0) & (df_imputed ['1stFlrSF'] <= 750.0) &  (df_imputed ['2ndFlrSF'] > 650.0) &(df_imputed ['2ndFlrSF'] <= 750.0),
   (df_imputed['2ndFlrSF']!= 0) &  (df_imputed ['1stFlrSF'] >= 750.0) & (df_imputed ['2ndFlrSF'] >= 750.0)
     ]
choicelist = [7.0, 8.0, 9.0, 10.0, 11.0]
df_imputed['Consumo kW'] = np.select(conditionlist2, choicelist, default= 0.0)

In [127]:
from sklearn.preprocessing import StandardScaler #Las features están rankiadas por orden según con el RFE utilizando las 50 más importantes
target = train['Consumo kW'].values

# select_features = [''MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street']

select_features = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street','Alley', 'LotShape', 'LandContour', 'Utilities','LandSlope', 'Neighborhood', 'BldgType', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
'MasVnrArea', 'ExterQual',  'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating','HeatingQC', 'CentralAir', 'Electrical',  '2ndFlrSF','GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath','HalfBath', 'BedroomAbvGr', 'KitchenQual','TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 
'GarageCars', 'GarageArea', 'GarageQual','GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF','EnclosedPorch','PoolArea', 'PoolQC','Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'SaleType','SalePrice']



scaler = StandardScaler()
dfScaled = scaler.fit_transform(df_imputed[select_features])

train = dfScaled[0:1460].copy()
test = dfScaled[1459:].copy()

In [128]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

#Regresión Lineal

In [129]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

clf = LinearRegression(normalize=True)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

print("El R2 es: {}".format(r2_score(y_test,y_pred)))
print("El pendiente es: {}".format(clf.coef_))
print("El intercept es: {}".format(clf.intercept_))

El R2 es: 0.7042454777074645
El pendiente es: [-2.21217258e-01 -3.33127812e-01  1.39493939e-01 -1.01710306e-02
  4.82178886e-03 -1.18245981e-01 -1.56947732e-01  8.71901885e-02
 -9.30094317e-04 -1.42705119e-02  9.23044476e-02  1.97375491e-01
  1.81586430e-01 -6.10392770e-02  4.05873242e-01  3.43112681e-02
  1.15988867e-01  7.59511314e-02 -1.07000289e-01  1.57700795e-01
 -1.03738632e-01  8.70443695e-02 -6.70356229e-02 -2.56736979e-04
  1.34714327e-01  1.45842632e-03 -1.95475641e-01 -2.47964373e+12
 -9.20648225e+11 -2.39201133e+12  2.39818178e+12  6.08789564e-03
 -1.07538967e-01 -2.27909876e-01  2.64380427e-02  3.80825982e+00
 -1.20745386e-01  5.79104529e-02 -4.01666206e-02 -2.05872029e-01
  2.21479930e-01 -6.35725098e-02  9.91878666e-02  3.15570459e-01
  1.19197010e-01 -1.96375508e-02  5.89296661e-02 -3.23055221e-01
  3.67876031e-01  1.60840618e-02 -1.46848613e-02 -5.63087019e-02
 -5.24805939e-02  1.23284827e-01  1.33982588e-02 -1.49492441e-01
 -1.25465098e-02 -1.60413315e-02 -5.40816401

#RandomForest Regressor

In [130]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


regressor = RandomForestRegressor(n_estimators=500, random_state=20)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

print("El R2 es: {}".format(r2_score(y_test,y_pred)))


El R2 es: 0.9245552850359153
